In [4]:
import cassandra;

print(cassandra.__version__)

3.25.0


In [17]:
!docker run --name cas1 -p 9042:9042 -e CASSANDRA_CLUSTER_NAME=MyCluster -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter1 -d cassandra

67444437e9e02d8f050775b7e1c36d8baf4c0b1828291f78c6a367095bcb71bc


In [26]:
!docker inspect --format='{{ .NetworkSettings.IPAddress }}' cas1

{ NetworkSettings.IPAddress }


In [30]:
# this ones were copied into terminal as it have $ sign here
!docker run --name cas2 -p 9142:9042 -e CASSANDRA_SEEDS="$(docker inspect --format='{{ .NetworkSettings.IPAddress }}' cas1)" -e CASSANDRA_CLUSTER_NAME=MyCluster -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter1 -d cassandra
 
!docker run --name cas3 -p 9242:9042 -e CASSANDRA_SEEDS="$(docker inspect --format='{{ .NetworkSettings.IPAddress }}' cas1)" -e CASSANDRA_CLUSTER_NAME=MyCluster -e CASSANDRA_ENDPOINT_SNITCH=GossipingPropertyFileSnitch -e CASSANDRA_DC=datacenter2 -d cassandra

3e41fb01350f5e6ad8b118de0a6c978b97fb73e5dcecbbffc5d4f1fd25172287
c0472c44fc4f2453b460390d3a2c2601cbb8b628b84abe45fbce582765b4e2ac


In [99]:
!docker exec -ti cas1 nodetool status
print("**")
!docker exec -ti cas2 nodetool status
print("**")
!docker exec -ti cas3 nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens       Owns    Host ID                               Rack
UN  172.17.0.3  340.42 KiB  256          ?       0d8112de-d0c4-48d8-9ad7-bfc3ba4906b2  rack1
UN  172.17.0.2  562.35 KiB  256          ?       8eb626dc-9dbb-4229-a5bc-00e19582a9fc  rack1
Datacenter: datacenter2
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens       Owns    Host ID                               Rack
UN  172.17.0.4  379.78 KiB  256          ?       e1a37d3f-7cb1-4050-a3b7-52b8f9f6d462  rack1

Note: Non-system keyspaces don't have the same replication settings, effective ownership information is meaningless
**
Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens       Owns    Host ID                               Rack
UN  172.17.0.3  340.42 KiB  256          ?       0d8112de-d0c4-48d8-9ad7-bfc3ba4906b2  rack1
U

In [37]:
def print_result_set(result_set):
    for row in result_set:
        print(row)

In [39]:
from cassandra.cluster import Cluster

# Викоритовуючи  cqlsh, створити три Keyspace з replication factor 1, 2, 3
cluster = Cluster()
session = cluster.connect()

session.execute("""
    CREATE KEYSPACE IF NOT EXISTS replicas1
    WITH replication = {
      'class' : 'SimpleStrategy',
      'replication_factor' : 1
    };
""")

session.execute("""
    CREATE KEYSPACE IF NOT EXISTS replicas2
    WITH replication = {
      'class' : 'SimpleStrategy',
      'replication_factor' : 2
    };
""")

session.execute("""
    CREATE KEYSPACE IF NOT EXISTS replicas3
    WITH replication = {
      'class' : 'SimpleStrategy',
      'replication_factor' : 3
    };
""")

In [46]:
# В кожному з кейспейсів створити таблиці 

session.set_keyspace("replicas1")

session.execute("""
        CREATE TABLE IF NOT EXISTS products (
        category text,
        producer text,
        price int,
        name text,
        PRIMARY KEY (category, producer, name)
        )
    """)

session.set_keyspace("replicas2")

session.execute("""
        CREATE TABLE IF NOT EXISTS products (
        category text,
        producer text,
        price int,
        name text,
        PRIMARY KEY (category, producer, name)
        )
    """)

session.set_keyspace("replicas3")

session.execute("""
        CREATE TABLE IF NOT EXISTS products (
        category text,
        producer text,
        price int,
        name text,
        PRIMARY KEY (category, producer, name)
        )
    """)

In [57]:
# Спробуйте писати і читати на / та з різних нод.
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT, ConsistencyLevel

profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.ONE, # by default in this drive consistency_level is LOCAL_ONE
)
cluster = Cluster(execution_profiles={EXEC_PROFILE_DEFAULT: profile})
session = cluster.connect()

session.set_keyspace("replicas1")

session.execute("""
        INSERT INTO products (category, producer, price, name) 
        VALUES ('phone', 'Apple', 500, 'iPhone3')
    """)

print_result_set(
    session.execute("""
        SELECT * FROM products
    """)
)

Row(category='phone', producer='Apple', name='iPhone3', price=500)


In [43]:
cluster = Cluster(port=9142)
session2 = cluster.connect()

session2.set_keyspace("replicas2")

session2.execute("""
        INSERT INTO products (category, producer, price, name) 
        VALUES ('phone', 'Apple', 600, 'iPhone4')
    """)

print_result_set(
    session2.execute("""
        SELECT * FROM products
    """)
)

Row(category='phone', producer='Apple', name='iPhone4', price=600)


In [48]:
cluster = Cluster(port=9242)
session3 = cluster.connect()

session3.set_keyspace("replicas3")

session3.execute("""
        INSERT INTO products (category, producer, price, name) 
        VALUES ('phone', 'Apple', 700, 'iPhone5')
    """)

print_result_set(
    session3.execute("""
        SELECT * FROM products
    """)
)

Row(category='phone', producer='Apple', name='iPhone5', price=700)


In [92]:
# Вставте дані в створені таблиці і подивіться на їх розподіл по вузлах кластера (для кожного з кейспесов - nodetool status)
!docker exec -ti cas1 nodetool status replicas1
print("**")
!docker exec -ti cas2 nodetool status replicas2
print("**")
!docker exec -ti cas3 nodetool status replicas3

Error response from daemon: Container 67444437e9e02d8f050775b7e1c36d8baf4c0b1828291f78c6a367095bcb71bc is not running
**
Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens       Owns (effective)  Host ID                               Rack
UN  172.17.0.3  265.34 KiB  256          64.3%             0d8112de-d0c4-48d8-9ad7-bfc3ba4906b2  rack1
DN  172.17.0.2  255.95 KiB  256          67.6%             8eb626dc-9dbb-4229-a5bc-00e19582a9fc  rack1
Datacenter: datacenter2
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens       Owns (effective)  Host ID                               Rack
DN  172.17.0.4  257.27 KiB  256          68.1%             e1a37d3f-7cb1-4050-a3b7-52b8f9f6d462  rack1

**
Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens       Owns (effective)  Host ID                               Rack
DN  172.17.0.3  190.2 KiB  25

In [66]:
# Для якогось запису з кожного з кейспейсу виведіть ноди на яких зберігаються дані
!docker exec -ti cas3 nodetool getendpoints replicas1 products phone
print("**")
!docker exec -ti cas3 nodetool getendpoints replicas2 products phone
print("**")
!docker exec -ti cas3 nodetool getendpoints replicas3 products phone


172.17.0.3
**
172.17.0.3
172.17.0.2
**
172.17.0.3
172.17.0.2
172.17.0.4


In [100]:
# Відключиіть одну з нод. 
# Для кожного з кейспейсів визначить з якими рівнями consistency можемо читати та писати, і які з них забезпечують strong consistency 
!docker stop cas1

cas1


In [95]:
profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.ONE,
)
cluster = Cluster(execution_profiles={EXEC_PROFILE_DEFAULT: profile}, port=9242)
session_one = cluster.connect()

session_one.set_keyspace("replicas1")

try:
    session_one.execute("""
            INSERT INTO products (category, producer, price, name) 
            VALUES ('TV', 'Samsung', 500, 'SmartTV')
        """)

    
    print_result_set(
        session_one.execute("""
            SELECT * FROM products
        """)
    )
except cassandra.cluster.NoHostAvailable as e:
    print(e)
    
session_one.set_keyspace("replicas2")


session_one.execute("""
        INSERT INTO products (category, producer, price, name) 
        VALUES ('TV', 'Samsung', 500, 'SmartTV')
    """)

print_result_set(
    session_one.execute("""
        SELECT * FROM products
    """)
)
    
session_one.set_keyspace("replicas3")

session_one.execute("""
        INSERT INTO products (category, producer, price, name) 
        VALUES ('TV', 'Samsung', 500, 'SmartTV')
    """)

print_result_set(
    session_one.execute("""
        SELECT * FROM products
    """)
)

# we can use consistent level one only for all keyspaces, but here we have error because all date for keyspace with 
# replication 1 is located on cas1 node

('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9242 datacenter2>: Unavailable('Error from server: code=1000 [Unavailable exception] message="Cannot achieve consistency level ONE" info={\'consistency\': \'ONE\', \'required_replicas\': 1, \'alive_replicas\': 0}')})
Row(category='phone', producer='Apple', name='iPhone4', price=600)
Row(category='TV', producer='Samsung', name='SmartTV', price=500)
Row(category='phone', producer='Apple', name='iPhone5', price=700)
Row(category='TV', producer='Samsung', name='SmartTV', price=500)


In [102]:
profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.TWO,
)
cluster = Cluster(execution_profiles={EXEC_PROFILE_DEFAULT: profile}, port=9242)
session_two = cluster.connect()

session_two.set_keyspace("replicas1")

try:
    session_two.execute("""
            INSERT INTO products (category, producer, price, name) 
            VALUES ('Player', 'Samsung', 500, 'MP3')
        """)

    
    print_result_set(
        session_two.execute("""
            SELECT * FROM products
        """)
    )
except cassandra.cluster.NoHostAvailable as e:
    print(e)
    
session_two.set_keyspace("replicas2")

try:
    session_two.execute("""
            INSERT INTO products (category, producer, price, name) 
            VALUES ('Player', 'Samsung', 500, 'MP3')
        """)
except cassandra.cluster.NoHostAvailable as e:
    print(e)

try:
    print_result_set(
        session_two.execute("""
            SELECT * FROM products
        """)
    )
except cassandra.cluster.NoHostAvailable as e:
    print(e)
    
session_two.set_keyspace("replicas3")


session_two.execute("""
        INSERT INTO products (category, producer, price, name) 
        VALUES ('Player', 'Samsung', 500, 'MP3')
    """)
    
print_result_set(
    session_two.execute("""
        SELECT * FROM products
    """)
)

# here we can use for keyspace with 3 replicas consistent level 2 and it will be strong consisstent for this keyspace
# no need to check consistent leve three as it will be faileing each time, because we have only two nodes right now

('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9242 datacenter2>: Unavailable('Error from server: code=1000 [Unavailable exception] message="Cannot achieve consistency level TWO" info={\'consistency\': \'TWO\', \'required_replicas\': 2, \'alive_replicas\': 0}')})
('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9242 datacenter2>: Unavailable('Error from server: code=1000 [Unavailable exception] message="Cannot achieve consistency level TWO" info={\'consistency\': \'TWO\', \'required_replicas\': 2, \'alive_replicas\': 1}')})
('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9242 datacenter2>: Unavailable('Error from server: code=1000 [Unavailable exception] message="Cannot achieve consistency level TWO" info={\'consistency\': \'TWO\', \'required_replicas\': 2, \'alive_replicas\': 1}')})
Row(category='phone', producer='Apple', name='iPhone5', price=700)
Row(category='TV', producer='Samsung', name='SmartTV', price=50

In [103]:
# Зробить так щоб три ноди працювали, але не бачили одна одну по мережі (відключити зв'язок між ними)
!docker start cas1
!docker network ls

cas1
NETWORK ID     NAME               DRIVER    SCOPE
a14d4743f4d7   bridge             bridge    local
d29f175808e6   host               host      local
ca94d58fbaa1   my-mongo-cluster   bridge    local
4eef352b9566   none               null      local


In [105]:
!docker network create cas1

8d6b3ac3b8de6af8a0fe0d9d2551ff54e00da9a6530380343ab0128ad4e5f639


In [106]:
!docker network create cas2

24ff336700a71b79538f4c0be1e083ab049f084f51b32058c24dfdb7e0e2be78


In [104]:
!docker network inspect bridge

[
    {
        "Name": "bridge",
        "Id": "a14d4743f4d7bac4d3e8c11af468c317da9de8e1c7e5b4efbb719389cca21464",
        "Created": "2021-04-04T14:44:30.759562629Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": [
                {
                    "Subnet": "172.17.0.0/16",
                    "Gateway": "172.17.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "41dfc795a1b5505a6a60a530c3de67350120501845217d77a8856ebaa39f9670": {
                "Name": "cas3",
                "EndpointID": "916b70de43e70a12268bc98056e7950d29ade6cb92d57f1fdf06808b69e445f3",
                "MacAddress": "02:42:ac:11:00:04",
                "IPv4Address": "1

In [124]:
!docker network disconnect bridge cas1
!docker network disconnect bridge cas2

In [125]:
!docker network connect cas1 cas1
!docker network connect cas2 cas2

In [126]:
!docker network inspect bridge

[
    {
        "Name": "bridge",
        "Id": "a14d4743f4d7bac4d3e8c11af468c317da9de8e1c7e5b4efbb719389cca21464",
        "Created": "2021-04-04T14:44:30.759562629Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": [
                {
                    "Subnet": "172.17.0.0/16",
                    "Gateway": "172.17.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "41dfc795a1b5505a6a60a530c3de67350120501845217d77a8856ebaa39f9670": {
                "Name": "cas3",
                "EndpointID": "916b70de43e70a12268bc98056e7950d29ade6cb92d57f1fdf06808b69e445f3",
                "MacAddress": "02:42:ac:11:00:04",
                "IPv4Address": "1

In [127]:
!docker network inspect cas1

[
    {
        "Name": "cas1",
        "Id": "8d6b3ac3b8de6af8a0fe0d9d2551ff54e00da9a6530380343ab0128ad4e5f639",
        "Created": "2021-04-04T17:53:18.0473522Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "172.19.0.0/16",
                    "Gateway": "172.19.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "67444437e9e02d8f050775b7e1c36d8baf4c0b1828291f78c6a367095bcb71bc": {
                "Name": "cas1",
                "EndpointID": "5f823917d60b7ecce5ac36ce591ddaf653985abb04f7a2c404befa3c8cfbfcdc",
                "MacAddress": "02:42:ac:13:00:02",
                "IPv4Address": "172.19.

In [128]:
!docker network inspect cas2

[
    {
        "Name": "cas2",
        "Id": "24ff336700a71b79538f4c0be1e083ab049f084f51b32058c24dfdb7e0e2be78",
        "Created": "2021-04-04T17:53:27.4798283Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "172.20.0.0/16",
                    "Gateway": "172.20.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "bafeb6446dfd66b0ef94378eb6bbfc3aeddc3f34d42df3dc206c6aa97fd85261": {
                "Name": "cas2",
                "EndpointID": "9341524a0267f9fa09391cb6ec6d322889718873862383887dfdd3cccbb3d55f",
                "MacAddress": "02:42:ac:14:00:02",
                "IPv4Address": "172.20.

In [129]:
# Для кейспейсу з replication factor 3 задайте рівень consistency рівним 1. 
# Виконайте запис одного й того самого значення, з однаковим primary key, але різними іншими значенням на кожну з нод 
# (тобто створіть конфлікт)

profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.ONE,
)
cluster = Cluster(execution_profiles={EXEC_PROFILE_DEFAULT: profile}, port=9042)
session = cluster.connect()

session.set_keyspace("replicas3")

session.execute("""
        INSERT INTO products (category, producer, price, name) 
        VALUES ('phone', 'Apple', 700, 'iPhone11')
    """)

In [130]:
profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.ONE,
)
cluster = Cluster(execution_profiles={EXEC_PROFILE_DEFAULT: profile}, port=9142)
session = cluster.connect()

session.set_keyspace("replicas3")

session.execute("""
        INSERT INTO products (category, producer, price, name) 
        VALUES ('phone', 'Apple', 800, 'iPhone11')
    """)

In [131]:
profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.ONE,
)
cluster = Cluster(execution_profiles={EXEC_PROFILE_DEFAULT: profile}, port=9242)
session = cluster.connect()

session.set_keyspace("replicas3")

session.execute("""
        INSERT INTO products (category, producer, price, name) 
        VALUES ('phone', 'Apple', 900, 'iPhone11')
    """)

In [132]:
# Об’єднайте ноди в кластер і визначте яке значення було прийнято кластером та за яким принципом
!docker network disconnect cas1 cas1
!docker network disconnect cas2 cas2

!docker network connect bridge cas1
!docker network connect bridge cas2

!docker network inspect bridge

[
    {
        "Name": "bridge",
        "Id": "a14d4743f4d7bac4d3e8c11af468c317da9de8e1c7e5b4efbb719389cca21464",
        "Created": "2021-04-04T14:44:30.759562629Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": [
                {
                    "Subnet": "172.17.0.0/16",
                    "Gateway": "172.17.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "41dfc795a1b5505a6a60a530c3de67350120501845217d77a8856ebaa39f9670": {
                "Name": "cas3",
                "EndpointID": "916b70de43e70a12268bc98056e7950d29ade6cb92d57f1fdf06808b69e445f3",
                "MacAddress": "02:42:ac:11:00:04",
                "IPv4Address": "1

In [133]:
print_result_set(session.execute("""
        SELECT * from products WHERE
        category='phone' and producer='Apple' and name='iPhone11'
    """))

Row(category='phone', producer='Apple', name='iPhone11', price=900)


In [ ]:
# write with  last timestamp won!

In [134]:
# Перевірте поведінку lightweight transactions для попередніх пунктів у розділеному на три частини кластері 
!docker network disconnect bridge cas1
!docker network disconnect bridge cas2

!docker network connect cas1 cas1
!docker network connect cas2 cas2

In [139]:
profile = ExecutionProfile(
    consistency_level=ConsistencyLevel.ONE,
)
cluster = Cluster(execution_profiles={EXEC_PROFILE_DEFAULT: profile}, port=9042)
session = cluster.connect()

session.set_keyspace("replicas3")

session.execute("""
        INSERT INTO products (category, producer, price, name) 
        VALUES ('phone', 'Apple', 700, 'iPhone12')
        IF NOT EXISTS;
    """)

NoHostAvailable: ('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9042 datacenter1>: Unavailable('Error from server: code=1000 [Unavailable exception] message="Cannot achieve consistency level SERIAL" info={\'consistency\': \'SERIAL\', \'required_replicas\': 2, \'alive_replicas\': 1}')})

In [ ]:
# Ми не можем виконати таку транзакцію так як неможливо досягнути рівня SERIAL який є аналогією QUORUM при реплікації 3